In [ ]:
#Problem Statement 

In [ ]:
#Related Work

In [ ]:
#Approach

Dataset Description:

About 300 products targeted for crowdsourcing manipulation are colledted from www.rapidworkers.com. Then, they are linked to Amazon website to collect their reviews which are about 21000 reviews (called root dataset). There are about 12000 unique users wrote a review. In the next step, we collect activity history of these users whihc are about 550000 reviews (called base dataset). In the next step, we adopt Mcauley dataset to get reviews of users who have co-reviewed an product with users in base dataset. The expanded dataset has about 11,000,000 reviews. 

In [526]:
!wget http://students.cse.tamu.edu/kaghazgaran/Dataset/AmazonDataset.zip

--2017-05-02 13:15:20--  http://students.cse.tamu.edu/kaghazgaran/Dataset/AmazonDataset.zip
Resolving students.cse.tamu.edu... 128.194.138.45
Connecting to students.cse.tamu.edu|128.194.138.45|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261141804 (249M) [application/zip]
Saving to: ‘AmazonDataset.zip’

AmazonDataset.zip   100%[===================>] 249.04M  4.83MB/s    in 59s     

2017-05-02 13:16:19 (4.24 MB/s) - ‘AmazonDataset.zip’ saved [261141804/261141804]



In [528]:
!unzip AmazonDataset.zip 

Archive:  AmazonDataset.zip
  inflating: AllinOneAddedText.json  
  inflating: CrowdturfingProds.txt   
  inflating: item_dedup_Sampled.csv  
  inflating: memberinfo.json         
  inflating: Total.json              


In [580]:
import json
from collections import defaultdict
import re
import datetime
import time


In [581]:
CrowdturfingProd=set()

In [582]:
with open('CrowdturfingProds.txt') as f:
    for line in f:
        line=line.strip()
        CrowdturfingProd.add(line)

In [583]:
print 'Number of Crowdturfing Items in the Dataset: ' + str(len(CrowdturfingProd))

Number of Crowdturfing Items in the Dataset: 281


In [584]:
#Converting item IDs and user IDs to integer values starting from zero

In [585]:
user=defaultdict()
item=defaultdict()
reviewId=set()
k_u=0
k_i=0
time_Min=1470791590
time_Max=0


In [586]:
#In the following four cells we read the required information from Amazon Datasets. 

In [587]:
#Root Dataset
with open('Total.json') as f:
    for line in f:
        nline=json.loads(line)
        
        if nline['reviewerId'] not in user:
            user[str(nline['reviewerId'])]=k_u
            k_u+=1
        if nline['prodId'] not in item:
            item[str(nline['prodId'])]=k_i
            k_i+=1
        reviewId.add(nline['reviewId'])    
            

In [588]:
print '.....Root Dataset Information.....'
print 'Number of users  :' + str(k_u)
print 'Number of itens  :' + str(k_i)
print 'Number of reviews:' + str(len(reviewId))


.....Root Dataset Information.....
Number of users  :11997
Number of itens  :272
Number of reviews:19122


In [589]:
#Base Dataset
with open('AllinOneAddedText.json') as f:
    for line in f:
        nline=json.loads(line)
       
        if nline['memberId'] not in user:
            user[str(nline['memberId'])]=k_u
            k_u+=1
            
        if nline['productId'] not in item:
            item[str(nline['productId'])]=k_i
            k_i+=1
            
        if int(nline['Date'])<time_Min:
            time_Min=int(nline['Date'])
        if int(nline['Date'])>time_Max:
            time_Max=int(nline['Date'])
            
        reviewId.add(nline['reviewId'])    
        
        
    

In [590]:
#Base Dataset
with open('memberinfo.json') as f:
    for line in f:
        nline=json.loads(line)
       
        
        if nline['member-id'] not in user:
            user[str(nline['member-id'])]=k_u
            k_u+=1
        if nline['product-id'] not in item:
            item[str(nline['product-id'])]=k_i
            k_i+=1
            
        if int(nline['time'])<time_Min:
            time_Min=int(nline['time'])
        if int(nline['time'])>time_Max:
            time_Max=int(nline['time'])
            
        reviewId.add(nline['review-id'])    

In [591]:
print '.....Base Dataset Information.....'
print 'Number of users  :' + str(k_u)
print 'Number of itens  :' + str(k_i)
print 'Number of reviews:' + str(len(reviewId))


.....Base Dataset Information.....
Number of users  :14387
Number of itens  :397556
Number of reviews:572786


In [592]:
#Expanded Dataset
with open('item_dedup_Sampled.csv', 'rb') as f:
        for row in f:
            temp=row.strip().split(',')
           
            
            if temp[0] not in user:
                user[str(temp[0])]=k_u
                k_u+=1
                
            if temp[1] not in item:
                item[str(temp[1])]=k_i
                k_i+=1
                
            if int(temp[3])<time_Min:
                time_Min=int(temp[3])
            if int(temp[3])>time_Max:
                time_Max=int(temp[3])
                
            

In [593]:
print '.....Expanded Dataset Information.....'
print 'Number of users  :' + str(k_u)
print 'Number of itens  :' + str(k_i)


.....Expanded Dataset Information.....
Number of users  :5887439
Number of itens  :442029


In [594]:
#preparing input file- #user,item,timestamp,rating,1

In [595]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [596]:
#this function converts timestamp to unix time
def ConvertTime(i):
    temp=i['reviewDate'].encode('utf-8')
    temp=re.split(r'\W+', temp)
    temp[1]=months.index(temp[1])+1
    dt = datetime.datetime(year=int(temp[3]), month=int(temp[1]), day=int(temp[2]))
    unix_dt=time.mktime(dt.timetuple())
    return unix_dt



In [597]:
date_format = "%Y/%m/%d"
#this function conver the unix time to days (weeks) starting from zero. zero is the minimum timestamp in the dataset
def ConvertUnixToDays(ut):
    b=datetime.datetime.fromtimestamp(ut).strftime('%Y-%m-%d %H:%M:%S')
    b=b.split()[0].replace('-','/')
    b=datetime.datetime.strptime(b, date_format)
    return b
    

In [598]:
#Starting time stamp
a=ConvertUnixToDays(int(time_Min))
print a

1996-12-15 00:00:00


In [604]:
reviewId=set()

In [605]:
with open ('AmazonData.txt','w') as input_file:
    with open('AllinOneAddedText.json') as f:
        for line in f:
            nline=json.loads(line)
            reviewId.add(nline['reviewId'])
            input_file.write(str(user[nline['memberId']]))
            input_file.write(',')
            
            try:
                input_file.write(str(item[nline['productId']]))
            except Exception:
                pass
            
            input_file.write(',')
            #....... for time ........
            b=ConvertUnixToDays(int(nline['Date']))
            delta = b - a
            input_file.write(str(delta.days/7))
            
            input_file.write(',')
            input_file.write(str(int(nline['Rate'])))
            input_file.write('\n')
        
    

In [606]:
with open ('AmazonData.txt','a+') as input_file:
    with open('Total.json') as f:
        for line in f:
            nline=json.loads(line)

            if nline['reviewId'] not in reviewId:


                reviewId.add(nline['reviewId'])

                input_file.write(str(user[nline['reviewerId']]))
                input_file.write(',')
                
                try:
                    input_file.write(str(item[nline['prodId']]))
                except Exception:
                    pass
                
                input_file.write(',')
                
                #.....For time.....
                tt=ConvertTime(nline)
                b=ConvertUnixToDays(int(tt))
                delta = b - a
                input_file.write(str(delta.days/7))
              
                
             
                input_file.write(',')
                input_file.write(str(int(float(nline['starRating']))))
                input_file.write('\n')


In [607]:
with open ('AmazonData.txt','a+') as input_file:
    with open('memberinfo.json') as f:
        for line in f:
            nline=json.loads(line)


            if nline['review-id'] not in reviewId:


                reviewId.add(nline['review-id'])

                input_file.write(str(user[nline['member-id']]))
                input_file.write(',')
                try:

                    input_file.write(str(item[nline['product-id']]))
                except Exception:
                    pass
                input_file.write(',')
                
                b=ConvertUnixToDays(int(nline['time']))
                delta = b - a
                input_file.write(str(delta.days/7))
                
                
                input_file.write(',')
                input_file.write(str(int(nline['rate'])))
                input_file.write('\n')



In [608]:
with open ('AmazonData.txt','a+') as input_file:
    with open('item_dedup_Sampled.csv', 'rb') as f:
            for row in f:
                temp=row.strip().split(',')
                input_file.write(str(user[temp[0]]))
                input_file.write(',')
                input_file.write(str(item[temp[1]]))
                input_file.write(',')

                b=ConvertUnixToDays(int(temp[3]))
                delta = b - a
                input_file.write(str(delta.days/7))
                
               

                input_file.write(',')
                input_file.write(str(int(float(temp[2]))))
                input_file.write('\n')
            
            
            

In [609]:
#Remove records with null values
with open('nAmazon.txt','w') as f:
    with open('AmazonData.txt') as ff:
        for row in ff:
            nff=row.strip().split(',')
            if '' not in nff:
                for i in nff:
                    f.write(i)
                    f.write(',')
                f.write('\n')    
            


In [610]:
!rm AmazonData.txt
!mv nAmazon.txt AmazonData.txt

In [611]:
#preparing the ground truth 

In [612]:
userProd=defaultdict(set)
verifiedUsrs=set()

In [613]:
with open('AllinOneAddedText.json') as f:
    for line in f:
        nline=json.loads(line)
        userProd[str(nline['memberId'])].add(str(nline['productId']))
        

In [615]:
with open('Total.json') as f:
    for line in f:
        nline=json.loads(line)
        
        userProd[str(nline['reviewerId'])].add(str(nline['prodId']))
        if (nline['verifiedPurchase']=='True'):
            verifiedUsrs.add(nline['reviewerId'])
        

In [617]:
with open('memberinfo.json') as f:
    for line in f:
        nline=json.loads(line)
        userProd[str(nline['member-id'])].add(str(nline['product-id']))
        

In [619]:
FraudulentUsrs=set()

In [620]:
#whoever wrote a review on crowdturfing items is malicious
for i in userProd:
    if len(userProd[i].intersection(CrowdturfingProd))>0 and i not in verifiedUsrs:
        FraudulentUsrs.add(i)
        

In [622]:
print 'Number of Fruadulent users in ground truth: '+ str(len(FraudulentUsrs))

Number of Fruadulent users in ground truth: 7502


In [624]:
#Execute decub on Amazon Dataset - arithmatic density measure
!java -cp dcube/target/dcube-1.0-SNAPSHOT.jar dcube.Proposed AmazonData.txt dcube/output/ari 3 ari density 100


input_path: AmazonData.txt
output_path: dcube/output/ari
dimension: 3
density_measure: ari
policy: density
num_of_blocks: 100

computing proper buffer size

storing the input tensor in the binary format...
Preprocess,3446

running the algorithm...

Block: 1
Volume: 52 X 60 X 16
Density: 237.609375
Mass: 10138
Block: 2
Volume: 8041 X 6272 X 1059
Density: 184.02088212334115
Mass: 942923
Block: 3
Volume: 41106 X 21790 X 1059
Density: 148.89639590336955
Mass: 3174223
Block: 4
Volume: 2446 X 583 X 168
Density: 85.53331248045042
Mass: 91150
Block: 5
Volume: 33250 X 11086 X 552
Density: 82.39600784173945
Mass: 1232864
Block: 6
Volume: 163437 X 51076 X 892
Density: 84.9101460040389
Mass: 6096690
Block: 7
Volume: 2848 X 388 X 85
Density: 52.47425474254743
Mass: 58089
Block: 8
Volume: 53815 X 9195 X 171
Density: 49.75647742200978
Mass: 1047888
Block: 9
Volume: 293143 X 44147 X 1059
Density: 50.36563134514954
Mass: 5680387
Block: 10
Volume: 3547 X 370 X 87
Density: 36.998001998001996
Mass: 49380


In [625]:
#Execute decub on Amazon Dataset - geometry density measure
!java -cp dcube/target/dcube-1.0-SNAPSHOT.jar dcube.Proposed AmazonData.txt dcube/output/geo 3 geo density 100



input_path: AmazonData.txt
output_path: dcube/output/geo
dimension: 3
density_measure: geo
policy: density
num_of_blocks: 100

computing proper buffer size

storing the input tensor in the binary format...
Preprocess,3226

running the algorithm...

Block: 1
Volume: 1099490 X 512 X 86
Density: 1920.5100225892459
Mass: 6999545
Block: 2
Volume: 3306224 X 11184 X 191
Density: 1366.769217672581
Mass: 26223025
Block: 3
Volume: 692444 X 17984 X 86
Density: 456.1510841065442
Mass: 4666936
Block: 4
Volume: 677 X 1 X 1
Density: 385.5042025019023
Mass: 3385
Block: 5
Volume: 145956 X 337 X 158
Density: 380.88171134912005
Mass: 754443
Block: 6
Volume: 1019277 X 25287 X 323
Density: 408.1506797714749
Mass: 8272140
Block: 7
Volume: 1430 X 1 X 2
Density: 502.8686164659068
Mass: 7138
Block: 8
Volume: 667 X 1 X 1
Density: 381.69859405898757
Mass: 3335
Block: 9
Volume: 601 X 1 X 1
Density: 356.08443111675433
Mass: 3005
Block: 10
Volume: 473354 X 75967 X 306
Density: 265.6288719208295
Mass: 5908167
Block:

In [626]:
user_2 = {y:x for x,y in user.iteritems()}


In [630]:
import glob
def Evaluation(s):
    path="dcube/output/"+str(s)+"/"
    allblocks=glob.glob(path+"*.tuples")
    for each_block in allblocks:
        dtctedusr=set()

        with open(each_block) as f:
            for row in f:

                nrow=row.strip().split(',')

                dtctedusr.add(user_2[int(nrow[0])])
            print each_block.replace(path,'')
            print "Detected users:                 "+str(len(dtctedusr))
            print "Intersection with Grounth Truth:"+str(len(dtctedusr.intersection(FraudulentUsrs)))
            print    
            
            
            


In [631]:
Evaluation('geo')

block_1.tuples
Detected users:                 1099490
Intersection with Grounth Truth:311

block_10.tuples
Detected users:                 473354
Intersection with Grounth Truth:6040

block_100.tuples
Detected users:                 1
Intersection with Grounth Truth:1

block_11.tuples
Detected users:                 372
Intersection with Grounth Truth:223

block_12.tuples
Detected users:                 2887
Intersection with Grounth Truth:1363

block_13.tuples
Detected users:                 342
Intersection with Grounth Truth:0

block_14.tuples
Detected users:                 912
Intersection with Grounth Truth:1

block_15.tuples
Detected users:                 1202
Intersection with Grounth Truth:1

block_16.tuples
Detected users:                 279
Intersection with Grounth Truth:0

block_17.tuples
Detected users:                 242
Intersection with Grounth Truth:0

block_18.tuples
Detected users:                 2083
Intersection with Grounth Truth:1

block_19.tuples
Detected 

In [633]:
Evaluation('ari')

block_1.tuples
Detected users:                 52
Intersection with Grounth Truth:44

block_10.tuples
Detected users:                 3546
Intersection with Grounth Truth:0

block_100.tuples
Detected users:                 164
Intersection with Grounth Truth:0

block_11.tuples
Detected users:                 50892
Intersection with Grounth Truth:0

block_12.tuples
Detected users:                 178825
Intersection with Grounth Truth:496

block_13.tuples
Detected users:                 16471
Intersection with Grounth Truth:0

block_14.tuples
Detected users:                 27293
Intersection with Grounth Truth:0

block_15.tuples
Detected users:                 192441
Intersection with Grounth Truth:0

block_16.tuples
Detected users:                 412031
Intersection with Grounth Truth:3333

block_17.tuples
Detected users:                 6112
Intersection with Grounth Truth:0

block_18.tuples
Detected users:                 9056
Intersection with Grounth Truth:0

block_19.tuples
Dete